In [ ]:
# Import required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.metrics import mean_squared_error,accuracy_score,mean_absolute_error
from sklearn.neural_network import MLPRegressor
from mlxtend.regressor import StackingRegressor
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.linear_model import ElasticNet, Lasso,Ridge, LinearRegression,HuberRegressor,SGDRegressor,TweedieRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import LinearSVR,SVR
from sklearn.metrics import mean_squared_error,explained_variance_score,r2_score
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor, AdaBoostRegressor,BaggingRegressor,ExtraTreesRegressor,VotingRegressor
import xgboost as xgb
import os
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.decomposition import PCA
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import f_regression

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Flatten,LSTM, GRU,Bidirectional,TimeDistributed, ConvLSTM2D
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.metrics import RootMeanSquaredError,mean_absolute_percentage_error
from keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dropout
from keras.models import Model
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [ ]:
from scipy.stats import t
from scipy.stats import norm
import scipy.stats as st

In [ ]:
# split a univariate time series into patterns
def get_Patterns(TSeries, n_inputs,h):
    X,y,z = pd.DataFrame(np.zeros((len(TSeries)-n_inputs-h+1,n_inputs))), pd.DataFrame(), pd.DataFrame()
    for i in range(len(TSeries)):
        # find the end of this pattern
        end_ix = i + n_inputs + h - 1
        # check if we are beyond the time series
        if end_ix > len(TSeries)-1:
            break
        # gather input and output parts of the pattern
        for j in range(n_inputs):
            X.loc[i,j]=TSeries.iloc[i+j,0]
        i=i+n_inputs
        y=y.append(TSeries.iloc[end_ix], ignore_index = True)
        sinX=pd.DataFrame(np.sin(X))
        cosX=pd.DataFrame(np.cos(X))
        squareX=pd.DataFrame(np.power(X,2))
        X1=pd.concat([X,sinX,cosX,squareX], axis=1)
    return np.array(X1),np.array(y)

In [ ]:
# originalData should be a Column Vectored DataFrame
def minmaxNorm(originalData, lenTrainValidation):
    max2norm=max(originalData.iloc[0:lenTrainValidation,0])
    min2norm=min(originalData.iloc[0:lenTrainValidation,0])
    lenOriginal=len(originalData)
    normalizedData=np.zeros(lenOriginal)
    normalizedData = []
    for i in range (lenOriginal):
        normalizedData.append((originalData.iloc[i]-min2norm)/(max2norm-min2norm))
    return pd.DataFrame(normalizedData)

In [ ]:
# originalData and forecastedData should be Column Vectored DataFrames
def minmaxDeNorm( originalData, forecastedData, lenTrainValidation):
    max2norm=max(originalData.iloc[0:lenTrainValidation,0])
    min2norm=min(originalData.iloc[0:lenTrainValidation,0])
    lenOriginal=len(originalData)
    denormalizedData=[]
    for i in range (lenOriginal):
        denormalizedData.append((forecastedData.iloc[i]*(max2norm-min2norm))+min2norm)
    return pd.DataFrame(denormalizedData)

In [ ]:
# Timeseries_Data and forecasted_value should be Column Vectored DataFrames
def findRMSE( Timeseries_Data, forecasted_value,lenTrainValidation):
    l=Timeseries_Data.shape[0]
    lenTest=l-lenTrainValidation
    trainRMSE=0;
    for i in range (lenTrainValidation):
        trainRMSE=trainRMSE+np.power((forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0]),2)
    trainRMSE=np.sqrt(trainRMSE/lenTrainValidation)

    testRMSE=0;
    for i in range (lenTrainValidation,l,1):
        testRMSE=testRMSE+np.power((forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0]),2)
    testRMSE=np.sqrt(testRMSE/lenTest)
    return trainRMSE, testRMSE

In [ ]:
# Timeseries_Data and forecasted_value should be Column Vectored DataFrames
def findSMAPE( Timeseries_Data, forecasted_value,lenTrainValidation):
    l=Timeseries_Data.shape[0]
    lenTest=l-lenTrainValidation
    trainSMAPE=0;
    for i in range (lenTrainValidation):
        trainSMAPE=trainSMAPE+(np.abs(forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0])/((np.abs(forecasted_value.iloc[i,0])+np.abs(Timeseries_Data.iloc[i,0]))/2))

    trainSMAPE=(trainSMAPE/(lenTrainValidation))*100;

    testSMAPE=0;
    for i in range (lenTrainValidation,l,1):
        testSMAPE=testSMAPE+(np.abs(forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0])/((np.abs(forecasted_value.iloc[i,0])+np.abs(Timeseries_Data.iloc[i,0]))/2))

    testSMAPE=(testSMAPE/lenTest)*100;
    return trainSMAPE, testSMAPE

In [ ]:
# Timeseries_Data and forecasted_value should be Column Vectored DataFrames
def findMAE( Timeseries_Data, forecasted_value,lenTrainValidation):
    l=Timeseries_Data.shape[0]
    lenTest=l-lenTrainValidation
    trainMAE=0;
    for i in range (lenTrainValidation):
        trainMAE=trainMAE+np.abs(forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0])
    trainMAE=(trainMAE/(lenTrainValidation));

    testMAE=0;
    for i in range (lenTrainValidation,l,1):
        testMAE=testMAE+np.abs(forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0])
    testMAE=(testMAE/lenTest);
    return trainMAE, testMAE

In [ ]:
# Timeseries_Data and forecasted_value should be Column Vectored DataFrames
def findMASE( Timeseries_Data, forecasted_value,lenTrainValidation):
    l=Timeseries_Data.shape[0]
    lenTest=l-lenTrainValidation
    trainMASE=0;
    T1=Timeseries_Data.iloc[0:lenTrainValidation,0]
    T2=T1.diff()
    T2=T2.drop(0)
    T2=np.abs(T2)
    M=np.mean(T2)
    for i in range (lenTrainValidation):
        trainMASE=trainMASE+(np.abs(forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0])/M)

    trainMASE=(trainMASE/(lenTrainValidation))

    testMASE=0
    T1=Timeseries_Data.iloc[lenTrainValidation:l,0]
    T2=T1.diff()
    T2=T2.drop(lenTrainValidation)
    T2=np.abs(T2)
    M=np.mean(T2)
    for i in range (lenTrainValidation,l,1):
        testMASE=testMASE+(np.abs(forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0])/M)
    testMASE=(testMASE/lenTest)
    return trainMASE,testMASE

In [ ]:
def Find_Fitness(x,y,lenValid,lenTest,model):
    NOP=y.shape[0]
    lenTrain=NOP-lenValid-lenTest
    xTrain=x[0:(lenTrain+lenValid),:]
    xValid=x[lenTrain:(lenTrain+lenValid),:]
    xTest=x[(lenTrain+lenValid):NOP,:]
    yTrain=y[0:(lenTrain+lenValid),0]
    yValid=y[lenTrain:(lenTrain+lenValid),0]
    yTest=y[(lenTrain+lenValid):NOP,0]
    model.fit(xTrain, yTrain)
    yhatNorm=model.predict(x).flatten().reshape(x.shape[0],1)
    return pd.DataFrame(yhatNorm)

In [ ]:
def find_confidence_interval_gaussian(confidenceLevel,sampleMean,sampleStandardError,error,ForecastedData):
    degrees_freedom = len(error)-1  #degree of freedom = sample size-1
    #sampleMean = np.mean(error)    #sample mean
    #sampleStandardError = st.sem(error)  #sample standard error
    confidenceInterval95 = st.norm.interval(confidence=confidenceLevel, loc=sampleMean, scale=sampleStandardError)
    lower95=pd.DataFrame(ForecastedData+confidenceInterval95[0])
    upper95=pd.DataFrame(ForecastedData+confidenceInterval95[1])
    df = pd.concat([lower95, upper95], axis = 1)
    return df

In [ ]:
def find_confidence_interval_t_location_scale(confidenceLevel,sampleMean,sampleStandardError,error,ForecastedData):
    degrees_freedom = len(error)-1  #degree of freedom = sample size-1
    #sampleMean = np.mean(error)    #sample mean
    #sampleStandardError = st.sem(error)  #sample standard error
    confidenceInterval95 = st.t.interval(confidence=confidenceLevel, df=error.shape[0]-1,loc=sampleMean, scale=sampleStandardError)
    lower95=pd.DataFrame(ForecastedData+confidenceInterval95[0])
    upper95=pd.DataFrame(ForecastedData+confidenceInterval95[1])
    df = pd.concat([lower95, upper95], axis = 1)
    return df

In [ ]:
def find_confidence_interval_beta(confidenceLevel,sampleMean,sampleStandardError,error,ForecastedData):
    degrees_freedom = len(error)-1  #degree of freedom = sample size-1
    #sampleMean = np.mean(error)    #sample mean
    #sampleStandardError = st.sem(error)  #sample standard error
    a, b = 2.31, 0.627
    confidenceInterval95 = st.beta.interval(confidence=confidenceLevel, a=a,b=b,loc=sampleMean, scale=sampleStandardError)
    lower95=pd.DataFrame(ForecastedData-confidenceInterval95[0])
    upper95=pd.DataFrame(ForecastedData+confidenceInterval95[1])
    df = pd.concat([lower95, upper95], axis = 1)
    return df

In [ ]:
def find_confidence_interval_cauchy(confidenceLevel,sampleMean,sampleStandardError,error,ForecastedData):
    degrees_freedom = len(error)-1  #degree of freedom = sample size-1
    #sampleMean = np.mean(error)    #sample mean
    #sampleStandardError = st.sem(error)  #sample standard error
    confidenceInterval95 = st.cauchy.interval(confidence=confidenceLevel, loc=sampleMean, scale=sampleStandardError)
    lower95=pd.DataFrame(ForecastedData+confidenceInterval95[0])
    upper95=pd.DataFrame(ForecastedData+confidenceInterval95[1])
    df = pd.concat([lower95, upper95], axis = 1)
    return df

In [ ]:
def find_confidence_interval_gamma(confidenceLevel,sampleMean,sampleStandardError,error,ForecastedData):
    degrees_freedom = len(error)-1  #degree of freedom = sample size-1
    #sampleMean = np.mean(error)    #sample mean
    #sampleStandardError = st.sem(error)  #sample standard error
    a=1.99
    confidenceInterval95 = st.gamma.interval(confidence=confidenceLevel, a=a,loc=sampleMean, scale=sampleStandardError)
    lower95=pd.DataFrame(ForecastedData-confidenceInterval95[0])
    upper95=pd.DataFrame(ForecastedData+confidenceInterval95[1])
    df = pd.concat([lower95, upper95], axis = 1)
    return df

In [ ]:
def find_confidence_interval_rayleigh(confidenceLevel,sampleMean,sampleStandardError,error,ForecastedData):
    degrees_freedom = len(error)-1  #degree of freedom = sample size-1
    #sampleMean = np.mean(error)    #sample mean
    #sampleStandardError = st.sem(error)  #sample standard error
    confidenceInterval95 = st.rayleigh.interval(confidence=confidenceLevel,loc=sampleMean, scale=sampleStandardError)
    lower95=pd.DataFrame(ForecastedData-confidenceInterval95[0])
    upper95=pd.DataFrame(ForecastedData+confidenceInterval95[1])
    df = pd.concat([lower95, upper95], axis = 1)
    return df

In [ ]:
def find_confidence_interval_uniform(confidenceLevel,sampleMean,sampleStandardError,error,ForecastedData):
    degrees_freedom = len(error)-1  #degree of freedom = sample size-1
    #sampleMean = np.mean(error)    #sample mean
    #sampleStandardError = st.sem(error)  #sample standard error
    confidenceInterval95 = st.uniform.interval(confidence=confidenceLevel,loc=sampleMean, scale=sampleStandardError)
    lower95=pd.DataFrame(ForecastedData-confidenceInterval95[0])
    upper95=pd.DataFrame(ForecastedData+confidenceInterval95[1])
    df = pd.concat([lower95, upper95], axis = 1)
    return df

In [ ]:
def find_confidence_interval_expon(confidenceLevel,sampleMean,sampleStandardError,error,ForecastedData):
    degrees_freedom = len(error)-1  #degree of freedom = sample size-1
    #sampleMean = np.mean(error)    #sample mean
    #sampleStandardError = st.sem(error)  #sample standard error
    confidenceInterval95 = st.expon.interval(confidence=confidenceLevel,loc=sampleMean, scale=sampleStandardError)
    lower95=pd.DataFrame(ForecastedData-confidenceInterval95[0])
    upper95=pd.DataFrame(ForecastedData+confidenceInterval95[1])
    df = pd.concat([lower95, upper95], axis = 1)
    return df

In [ ]:
def find_confidence_interval_laplace(confidenceLevel,sampleMean,sampleStandardError,error,ForecastedData):
    degrees_freedom = len(error)-1  #degree of freedom = sample size-1
    #sampleMean = np.mean(error)    #sample mean
    #sampleStandardError = st.sem(error)  #sample standard error
    confidenceInterval95 = st.laplace.interval(confidence=confidenceLevel,loc=sampleMean, scale=sampleStandardError)
    lower95=pd.DataFrame(ForecastedData+confidenceInterval95[0])
    upper95=pd.DataFrame(ForecastedData+confidenceInterval95[1])
    df = pd.concat([lower95, upper95], axis = 1)
    return df

In [ ]:
def find_confidence_interval_maxwell(confidenceLevel,sampleMean,sampleStandardError,error,ForecastedData):
    degrees_freedom = len(error)-1  #degree of freedom = sample size-1
    #sampleMean = np.mean(error)    #sample mean
    #sampleStandardError = st.sem(error)  #sample standard error
    confidenceInterval95 = st.maxwell.interval(confidence=confidenceLevel,loc=sampleMean, scale=sampleStandardError)
    lower95=pd.DataFrame(ForecastedData-confidenceInterval95[0])
    upper95=pd.DataFrame(ForecastedData+confidenceInterval95[1])
    df = pd.concat([lower95, upper95], axis = 1)
    return df

In [ ]:
def find_Prediction_Interval_Evaluation(pi,Original,Forecasted,ci):
    # Computation of Prediction Interval Coverage Probability (PICP)
    count=0
    cc=pd.DataFrame()
    for i in range(0,Original.shape[0],1):
        dat=Original.iloc[i,0]
        if dat >= pi.iloc[i,0] and dat <= pi.iloc[i,1]:
            count+=1
    PICP=(1/Original.shape[0])*count*100

    # Computation of Prediction Interval Normalized Average Width (PINAW)
    sum=0
    for i in range(0,pi.shape[0],1):
        sum+=pi.iloc[i,1]-pi.iloc[i,0]
    A=np.max(Original)-np.min(Original)
    PINAW=(1/(A*pi.shape[0]))*sum

    # Computation of Accumulated Width Deviation (AWD)
    sum=0
    for i in range(0,Original.shape[0],1):
        dat=Original.iloc[i,0]
        if dat >= pi.iloc[i,0] and dat <= pi.iloc[i,1]:
            sum+=0
        elif dat < pi.iloc[i,0]:
            sum+=(pi.iloc[i,0]-dat)/(pi.iloc[i,1]-pi.iloc[i,0])
        else:
            sum+=(dat-pi.iloc[i,1])/(pi.iloc[i,1]-pi.iloc[i,0])
    AWD=(1/Original.shape[0])*sum

    # Computation of Average Coverage Error (ACE)=PICP-PINC
    ACE=PICP-ci

    return PICP, PINAW, AWD,ACE

In [ ]:
def gaussian(Original,ForecastedData,error,filename,modelname,Scale):
    #Calculate different confidence intervals with Gaussian Distribution
    location=0
    conf95=find_confidence_interval_gaussian(0.95,location,Scale,error,ForecastedData)
    conf90=find_confidence_interval_gaussian(0.90,location,Scale,error,ForecastedData)
    conf80=find_confidence_interval_gaussian(0.80,location,Scale,error,ForecastedData)
    conf70=find_confidence_interval_gaussian(0.70,location,Scale,error,ForecastedData)
    conf60=find_confidence_interval_gaussian(0.60,location,Scale,error,ForecastedData)
    IF = pd.concat([conf95, conf90, conf80, conf70, conf60], axis=1)
    fig, ax = plt.subplots(1,1)
    fig.set_figheight(5)
    fig.set_figwidth(10)
    ax.fill_between(conf95.index,conf95.iloc[:,0],conf95.iloc[:,1], color='#ff0000',label='95%')
    ax.fill_between(conf90.index,conf90.iloc[:,0],conf90.iloc[:,1], color='#cf0000',label='90%')
    ax.fill_between(conf80.index,conf80.iloc[:,0],conf80.iloc[:,1], color='#af0000',label='80%')
    ax.fill_between(conf90.index,conf70.iloc[:,0],conf70.iloc[:,1], color='#8f0000',label='70%')
    ax.fill_between(conf80.index,conf60.iloc[:,0],conf60.iloc[:,1], color='#6f0000',label='60%')
    ax.plot(ForecastedData, linewidth=3,label='Forecasts')
    ax.plot(Original, linewidth=3,label='True Values',color='black', ls=':')
    ax.legend(loc="best")
    ax.set(xlabel='Time / Month', ylabel=file_name)
    PIEvaluation=pd.DataFrame(np.zeros((5,4)))
    PIEvaluation.iloc[0,0], PIEvaluation.iloc[0,1], PIEvaluation.iloc[0,2], PIEvaluation.iloc[0,3]=find_Prediction_Interval_Evaluation(conf95,Original,ForecastedData,95)
    PIEvaluation.iloc[1,0], PIEvaluation.iloc[1,1], PIEvaluation.iloc[1,2], PIEvaluation.iloc[1,3]=find_Prediction_Interval_Evaluation(conf90,Original,ForecastedData,90)
    PIEvaluation.iloc[2,0], PIEvaluation.iloc[2,1], PIEvaluation.iloc[2,2], PIEvaluation.iloc[2,3]=find_Prediction_Interval_Evaluation(conf80,Original,ForecastedData,80)
    PIEvaluation.iloc[3,0], PIEvaluation.iloc[3,1], PIEvaluation.iloc[3,2], PIEvaluation.iloc[3,3]=find_Prediction_Interval_Evaluation(conf70,Original,ForecastedData,70)
    PIEvaluation.iloc[4,0], PIEvaluation.iloc[4,1], PIEvaluation.iloc[4,2], PIEvaluation.iloc[4,3]=find_Prediction_Interval_Evaluation(conf60,Original,ForecastedData,60)
    img_name=filename+"_"+modelname+"Gaussian_Distribution_Prediction_Interval.png"
    plt.savefig(img_name)
    return IF,PIEvaluation

In [ ]:
def t_location_scale(Original,ForecastedData,error,filename,modelname,Scale):
    #Calculate different confidence intervals with t_location_scale Distribution
    location=0
    conf95=find_confidence_interval_t_location_scale(0.95,location,Scale,error,ForecastedData)
    conf90=find_confidence_interval_t_location_scale(0.90,location,Scale,error,ForecastedData)
    conf80=find_confidence_interval_t_location_scale(0.80,location,Scale,error,ForecastedData)
    conf70=find_confidence_interval_t_location_scale(0.70,location,Scale,error,ForecastedData)
    conf60=find_confidence_interval_t_location_scale(0.60,location,Scale,error,ForecastedData)
    IF = pd.concat([conf95, conf90, conf80, conf70, conf60], axis=1)
    fig, ax = plt.subplots(1,1)
    fig.set_figheight(5)
    fig.set_figwidth(10)
    ax.fill_between(conf95.index,conf95.iloc[:,0],conf95.iloc[:,1], color='#ff0000',label='95%')
    ax.fill_between(conf90.index,conf90.iloc[:,0],conf90.iloc[:,1], color='#cf0000',label='90%')
    ax.fill_between(conf80.index,conf80.iloc[:,0],conf80.iloc[:,1], color='#af0000',label='80%')
    ax.fill_between(conf90.index,conf70.iloc[:,0],conf70.iloc[:,1], color='#8f0000',label='70%')
    ax.fill_between(conf80.index,conf60.iloc[:,0],conf60.iloc[:,1], color='#6f0000',label='60%')
    ax.plot(ForecastedData, linewidth=3,label='Forecasts')
    ax.plot(Original, linewidth=3,label='True Values',color='black', ls=':')
    ax.legend(loc="best")
    ax.set(xlabel='Time / Month', ylabel=filename)
    PIEvaluation=pd.DataFrame(np.zeros((5,4)))
    PIEvaluation.iloc[0,0], PIEvaluation.iloc[0,1], PIEvaluation.iloc[0,2], PIEvaluation.iloc[0,3]=find_Prediction_Interval_Evaluation(conf95,Original,ForecastedData,95)
    PIEvaluation.iloc[1,0], PIEvaluation.iloc[1,1], PIEvaluation.iloc[1,2], PIEvaluation.iloc[1,3]=find_Prediction_Interval_Evaluation(conf90,Original,ForecastedData,90)
    PIEvaluation.iloc[2,0], PIEvaluation.iloc[2,1], PIEvaluation.iloc[2,2], PIEvaluation.iloc[2,3]=find_Prediction_Interval_Evaluation(conf80,Original,ForecastedData,80)
    PIEvaluation.iloc[3,0], PIEvaluation.iloc[3,1], PIEvaluation.iloc[3,2], PIEvaluation.iloc[3,3]=find_Prediction_Interval_Evaluation(conf70,Original,ForecastedData,70)
    PIEvaluation.iloc[4,0], PIEvaluation.iloc[4,1], PIEvaluation.iloc[4,2], PIEvaluation.iloc[4,3]=find_Prediction_Interval_Evaluation(conf60,Original,ForecastedData,60)
    img_name=filename+"_"+modelname+"t_Location_Scale_Distribution_Prediction_Interval.png"
    plt.savefig(img_name)
    return IF,PIEvaluation

In [ ]:
def Beta(Original,ForecastedData,error,filename,modelname,Scale):
    #Calculate different confidence intervals with Beta Distribution
    location=0
    conf95=find_confidence_interval_beta(0.95,location,Scale,error,ForecastedData)
    conf90=find_confidence_interval_beta(0.90,location,Scale,error,ForecastedData)
    conf80=find_confidence_interval_beta(0.80,location,Scale,error,ForecastedData)
    conf70=find_confidence_interval_beta(0.70,location,Scale,error,ForecastedData)
    conf60=find_confidence_interval_beta(0.60,location,Scale,error,ForecastedData)
    IF = pd.concat([conf95, conf90, conf80, conf70, conf60], axis=1)
    fig, ax = plt.subplots(1,1)
    fig.set_figheight(5)
    fig.set_figwidth(10)
    ax.fill_between(conf95.index,conf95.iloc[:,0],conf95.iloc[:,1], color='#ff0000',label='95%')
    ax.fill_between(conf90.index,conf90.iloc[:,0],conf90.iloc[:,1], color='#cf0000',label='90%')
    ax.fill_between(conf80.index,conf80.iloc[:,0],conf80.iloc[:,1], color='#af0000',label='80%')
    ax.fill_between(conf90.index,conf70.iloc[:,0],conf70.iloc[:,1], color='#8f0000',label='70%')
    ax.fill_between(conf80.index,conf60.iloc[:,0],conf60.iloc[:,1], color='#6f0000',label='60%')
    ax.plot(ForecastedData, linewidth=3,label='Forecasts')
    ax.plot(Original, linewidth=3,label='True Values',color='black', ls=':')
    ax.legend(loc="best")
    ax.set(xlabel='Time / Month', ylabel=filename)
    PIEvaluation=pd.DataFrame(np.zeros((5,4)))
    PIEvaluation.iloc[0,0], PIEvaluation.iloc[0,1], PIEvaluation.iloc[0,2], PIEvaluation.iloc[0,3]=find_Prediction_Interval_Evaluation(conf95,Original,ForecastedData,95)
    PIEvaluation.iloc[1,0], PIEvaluation.iloc[1,1], PIEvaluation.iloc[1,2], PIEvaluation.iloc[1,3]=find_Prediction_Interval_Evaluation(conf90,Original,ForecastedData,90)
    PIEvaluation.iloc[2,0], PIEvaluation.iloc[2,1], PIEvaluation.iloc[2,2], PIEvaluation.iloc[2,3]=find_Prediction_Interval_Evaluation(conf80,Original,ForecastedData,80)
    PIEvaluation.iloc[3,0], PIEvaluation.iloc[3,1], PIEvaluation.iloc[3,2], PIEvaluation.iloc[3,3]=find_Prediction_Interval_Evaluation(conf70,Original,ForecastedData,70)
    PIEvaluation.iloc[4,0], PIEvaluation.iloc[4,1], PIEvaluation.iloc[4,2], PIEvaluation.iloc[4,3]=find_Prediction_Interval_Evaluation(conf60,Original,ForecastedData,60)
    img_name=filename+"_"+modelname+"Beta_Distribution_Prediction_Interval.png"
    plt.savefig(img_name)
    return IF,PIEvaluation

In [ ]:
def Cauchy(Original,ForecastedData,error,filename,modelname,Scale):
    #Calculate different confidence intervals with Cauchy Distribution
    location=0
    conf95=find_confidence_interval_cauchy(0.95,location,Scale,error,ForecastedData)
    conf90=find_confidence_interval_cauchy(0.90,location,Scale,error,ForecastedData)
    conf80=find_confidence_interval_cauchy(0.80,location,Scale,error,ForecastedData)
    conf70=find_confidence_interval_cauchy(0.70,location,Scale,error,ForecastedData)
    conf60=find_confidence_interval_cauchy(0.60,location,Scale,error,ForecastedData)
    IF = pd.concat([conf95, conf90, conf80, conf70, conf60], axis=1)
    fig, ax = plt.subplots(1,1)
    fig.set_figheight(5)
    fig.set_figwidth(10)
    ax.fill_between(conf95.index,conf95.iloc[:,0],conf95.iloc[:,1], color='#ff0000',label='95%')
    ax.fill_between(conf90.index,conf90.iloc[:,0],conf90.iloc[:,1], color='#cf0000',label='90%')
    ax.fill_between(conf80.index,conf80.iloc[:,0],conf80.iloc[:,1], color='#af0000',label='80%')
    ax.fill_between(conf90.index,conf70.iloc[:,0],conf70.iloc[:,1], color='#8f0000',label='70%')
    ax.fill_between(conf80.index,conf60.iloc[:,0],conf60.iloc[:,1], color='#6f0000',label='60%')
    ax.plot(ForecastedData, linewidth=3,label='Forecasts')
    ax.plot(Original, linewidth=3,label='True Values',color='black', ls=':')
    ax.legend(loc="best")
    ax.set(xlabel='Time / Month', ylabel=filename)
    PIEvaluation=pd.DataFrame(np.zeros((5,4)))
    PIEvaluation.iloc[0,0], PIEvaluation.iloc[0,1], PIEvaluation.iloc[0,2], PIEvaluation.iloc[0,3]=find_Prediction_Interval_Evaluation(conf95,Original,ForecastedData,95)
    PIEvaluation.iloc[1,0], PIEvaluation.iloc[1,1], PIEvaluation.iloc[1,2], PIEvaluation.iloc[1,3]=find_Prediction_Interval_Evaluation(conf90,Original,ForecastedData,90)
    PIEvaluation.iloc[2,0], PIEvaluation.iloc[2,1], PIEvaluation.iloc[2,2], PIEvaluation.iloc[2,3]=find_Prediction_Interval_Evaluation(conf80,Original,ForecastedData,80)
    PIEvaluation.iloc[3,0], PIEvaluation.iloc[3,1], PIEvaluation.iloc[3,2], PIEvaluation.iloc[3,3]=find_Prediction_Interval_Evaluation(conf70,Original,ForecastedData,70)
    PIEvaluation.iloc[4,0], PIEvaluation.iloc[4,1], PIEvaluation.iloc[4,2], PIEvaluation.iloc[4,3]=find_Prediction_Interval_Evaluation(conf60,Original,ForecastedData,60)
    img_name=filename+"_"+modelname+"Cauchy_Distribution_Prediction_Interval.png"
    plt.savefig(img_name)
    return IF,PIEvaluation

In [ ]:
def Gamma(Original,ForecastedData,error,filename,modelname,Scale):
    #Calculate different confidence intervals with Gamma Distribution
    location=0
    conf95=find_confidence_interval_gamma(0.95,location,Scale,error,ForecastedData)
    conf90=find_confidence_interval_gamma(0.90,location,Scale,error,ForecastedData)
    conf80=find_confidence_interval_gamma(0.80,location,Scale,error,ForecastedData)
    conf70=find_confidence_interval_gamma(0.70,location,Scale,error,ForecastedData)
    conf60=find_confidence_interval_gamma(0.60,location,Scale,error,ForecastedData)
    IF = pd.concat([conf95, conf90, conf80, conf70, conf60], axis=1)
    fig, ax = plt.subplots(1,1)
    fig.set_figheight(5)
    fig.set_figwidth(10)
    ax.fill_between(conf95.index,conf95.iloc[:,0],conf95.iloc[:,1], color='#ff0000',label='95%')
    ax.fill_between(conf90.index,conf90.iloc[:,0],conf90.iloc[:,1], color='#cf0000',label='90%')
    ax.fill_between(conf80.index,conf80.iloc[:,0],conf80.iloc[:,1], color='#af0000',label='80%')
    ax.fill_between(conf90.index,conf70.iloc[:,0],conf70.iloc[:,1], color='#8f0000',label='70%')
    ax.fill_between(conf80.index,conf60.iloc[:,0],conf60.iloc[:,1], color='#6f0000',label='60%')
    ax.plot(ForecastedData, linewidth=3,label='Forecasts')
    ax.plot(Original, linewidth=3,label='True Values',color='black', ls=':')
    ax.legend(loc="best")
    ax.set(xlabel='Time / Month', ylabel=filename)
    PIEvaluation=pd.DataFrame(np.zeros((5,4)))
    PIEvaluation.iloc[0,0], PIEvaluation.iloc[0,1], PIEvaluation.iloc[0,2], PIEvaluation.iloc[0,3]=find_Prediction_Interval_Evaluation(conf95,Original,ForecastedData,95)
    PIEvaluation.iloc[1,0], PIEvaluation.iloc[1,1], PIEvaluation.iloc[1,2], PIEvaluation.iloc[1,3]=find_Prediction_Interval_Evaluation(conf90,Original,ForecastedData,90)
    PIEvaluation.iloc[2,0], PIEvaluation.iloc[2,1], PIEvaluation.iloc[2,2], PIEvaluation.iloc[2,3]=find_Prediction_Interval_Evaluation(conf80,Original,ForecastedData,80)
    PIEvaluation.iloc[3,0], PIEvaluation.iloc[3,1], PIEvaluation.iloc[3,2], PIEvaluation.iloc[3,3]=find_Prediction_Interval_Evaluation(conf70,Original,ForecastedData,70)
    PIEvaluation.iloc[4,0], PIEvaluation.iloc[4,1], PIEvaluation.iloc[4,2], PIEvaluation.iloc[4,3]=find_Prediction_Interval_Evaluation(conf60,Original,ForecastedData,60)
    img_name=filename+"_"+modelname+"Gamma_Distribution_Prediction_Interval.png"
    plt.savefig(img_name)
    return IF,PIEvaluation

In [ ]:
def Rayleigh(Original,ForecastedData,error,filename,modelname,Scale):
    #Calculate different confidence intervals with Rayleigh Distribution
    location=0
    conf95=find_confidence_interval_rayleigh(0.95,location,Scale,error,ForecastedData)
    conf90=find_confidence_interval_rayleigh(0.90,location,Scale,error,ForecastedData)
    conf80=find_confidence_interval_rayleigh(0.80,location,Scale,error,ForecastedData)
    conf70=find_confidence_interval_rayleigh(0.70,location,Scale,error,ForecastedData)
    conf60=find_confidence_interval_rayleigh(0.60,location,Scale,error,ForecastedData)
    IF = pd.concat([conf95, conf90, conf80, conf70, conf60], axis=1)
    fig, ax = plt.subplots(1,1)
    fig.set_figheight(5)
    fig.set_figwidth(10)
    ax.fill_between(conf95.index,conf95.iloc[:,0],conf95.iloc[:,1], color='#ff0000',label='95%')
    ax.fill_between(conf90.index,conf90.iloc[:,0],conf90.iloc[:,1], color='#cf0000',label='90%')
    ax.fill_between(conf80.index,conf80.iloc[:,0],conf80.iloc[:,1], color='#af0000',label='80%')
    ax.fill_between(conf90.index,conf70.iloc[:,0],conf70.iloc[:,1], color='#8f0000',label='70%')
    ax.fill_between(conf80.index,conf60.iloc[:,0],conf60.iloc[:,1], color='#6f0000',label='60%')
    ax.plot(ForecastedData, linewidth=3,label='Forecasts')
    ax.plot(Original, linewidth=3,label='True Values',color='black', ls=':')
    ax.legend(loc="best")
    ax.set(xlabel='Time / Month', ylabel=filename)
    PIEvaluation=pd.DataFrame(np.zeros((5,4)))
    PIEvaluation.iloc[0,0], PIEvaluation.iloc[0,1], PIEvaluation.iloc[0,2], PIEvaluation.iloc[0,3]=find_Prediction_Interval_Evaluation(conf95,Original,ForecastedData,95)
    PIEvaluation.iloc[1,0], PIEvaluation.iloc[1,1], PIEvaluation.iloc[1,2], PIEvaluation.iloc[1,3]=find_Prediction_Interval_Evaluation(conf90,Original,ForecastedData,90)
    PIEvaluation.iloc[2,0], PIEvaluation.iloc[2,1], PIEvaluation.iloc[2,2], PIEvaluation.iloc[2,3]=find_Prediction_Interval_Evaluation(conf80,Original,ForecastedData,80)
    PIEvaluation.iloc[3,0], PIEvaluation.iloc[3,1], PIEvaluation.iloc[3,2], PIEvaluation.iloc[3,3]=find_Prediction_Interval_Evaluation(conf70,Original,ForecastedData,70)
    PIEvaluation.iloc[4,0], PIEvaluation.iloc[4,1], PIEvaluation.iloc[4,2], PIEvaluation.iloc[4,3]=find_Prediction_Interval_Evaluation(conf60,Original,ForecastedData,60)
    img_name=filename+"_"+modelname+"Rayleigh_Distribution_Prediction_Interval.png"
    plt.savefig(img_name)
    return IF,PIEvaluation

In [ ]:
def Uniform(Original,ForecastedData,error,filename,modelname,Scale):
    #Calculate different confidence intervals with Uniform Distribution
    location=0
    conf95=find_confidence_interval_uniform(0.95,location,Scale,error,ForecastedData)
    conf90=find_confidence_interval_uniform(0.90,location,Scale,error,ForecastedData)
    conf80=find_confidence_interval_uniform(0.80,location,Scale,error,ForecastedData)
    conf70=find_confidence_interval_uniform(0.70,location,Scale,error,ForecastedData)
    conf60=find_confidence_interval_uniform(0.60,location,Scale,error,ForecastedData)
    IF = pd.concat([conf95, conf90, conf80, conf70, conf60], axis=1)
    fig, ax = plt.subplots(1,1)
    fig.set_figheight(5)
    fig.set_figwidth(10)
    ax.fill_between(conf95.index,conf95.iloc[:,0],conf95.iloc[:,1], color='#ff0000',label='95%')
    ax.fill_between(conf90.index,conf90.iloc[:,0],conf90.iloc[:,1], color='#cf0000',label='90%')
    ax.fill_between(conf80.index,conf80.iloc[:,0],conf80.iloc[:,1], color='#af0000',label='80%')
    ax.fill_between(conf90.index,conf70.iloc[:,0],conf70.iloc[:,1], color='#8f0000',label='70%')
    ax.fill_between(conf80.index,conf60.iloc[:,0],conf60.iloc[:,1], color='#6f0000',label='60%')
    ax.plot(ForecastedData, linewidth=3,label='Forecasts')
    ax.plot(Original, linewidth=3,label='True Values',color='black', ls=':')
    ax.legend(loc="best")
    ax.set(xlabel='Time / Month', ylabel=filename)
    PIEvaluation=pd.DataFrame(np.zeros((5,4)))
    PIEvaluation.iloc[0,0], PIEvaluation.iloc[0,1], PIEvaluation.iloc[0,2], PIEvaluation.iloc[0,3]=find_Prediction_Interval_Evaluation(conf95,Original,ForecastedData,95)
    PIEvaluation.iloc[1,0], PIEvaluation.iloc[1,1], PIEvaluation.iloc[1,2], PIEvaluation.iloc[1,3]=find_Prediction_Interval_Evaluation(conf90,Original,ForecastedData,90)
    PIEvaluation.iloc[2,0], PIEvaluation.iloc[2,1], PIEvaluation.iloc[2,2], PIEvaluation.iloc[2,3]=find_Prediction_Interval_Evaluation(conf80,Original,ForecastedData,80)
    PIEvaluation.iloc[3,0], PIEvaluation.iloc[3,1], PIEvaluation.iloc[3,2], PIEvaluation.iloc[3,3]=find_Prediction_Interval_Evaluation(conf70,Original,ForecastedData,70)
    PIEvaluation.iloc[4,0], PIEvaluation.iloc[4,1], PIEvaluation.iloc[4,2], PIEvaluation.iloc[4,3]=find_Prediction_Interval_Evaluation(conf60,Original,ForecastedData,60)
    img_name=filename+"_"+modelname+"Uniform_Distribution_Prediction_Interval.png"
    plt.savefig(img_name)
    return IF,PIEvaluation

In [ ]:
def Exponential(Original,ForecastedData,error,filename,modelname,Scale):
    #Calculate different confidence intervals with Exponential Distribution
    location=0
    conf95=find_confidence_interval_expon(0.95,location,Scale,error,ForecastedData)
    conf90=find_confidence_interval_expon(0.90,location,Scale,error,ForecastedData)
    conf80=find_confidence_interval_expon(0.80,location,Scale,error,ForecastedData)
    conf70=find_confidence_interval_expon(0.70,location,Scale,error,ForecastedData)
    conf60=find_confidence_interval_expon(0.60,location,Scale,error,ForecastedData)
    IF = pd.concat([conf95, conf90, conf80, conf70, conf60], axis=1)
    fig, ax = plt.subplots(1,1)
    fig.set_figheight(5)
    fig.set_figwidth(10)
    ax.fill_between(conf95.index,conf95.iloc[:,0],conf95.iloc[:,1], color='#ff0000',label='95%')
    ax.fill_between(conf90.index,conf90.iloc[:,0],conf90.iloc[:,1], color='#cf0000',label='90%')
    ax.fill_between(conf80.index,conf80.iloc[:,0],conf80.iloc[:,1], color='#af0000',label='80%')
    ax.fill_between(conf90.index,conf70.iloc[:,0],conf70.iloc[:,1], color='#8f0000',label='70%')
    ax.fill_between(conf80.index,conf60.iloc[:,0],conf60.iloc[:,1], color='#6f0000',label='60%')
    ax.plot(ForecastedData, linewidth=3,label='Forecasts')
    ax.plot(Original, linewidth=3,label='True Values',color='black', ls=':')
    ax.legend(loc="best")
    ax.set(xlabel='Time / Month', ylabel=filename)
    PIEvaluation=pd.DataFrame(np.zeros((5,4)))
    PIEvaluation.iloc[0,0], PIEvaluation.iloc[0,1], PIEvaluation.iloc[0,2], PIEvaluation.iloc[0,3]=find_Prediction_Interval_Evaluation(conf95,Original,ForecastedData,95)
    PIEvaluation.iloc[1,0], PIEvaluation.iloc[1,1], PIEvaluation.iloc[1,2], PIEvaluation.iloc[1,3]=find_Prediction_Interval_Evaluation(conf90,Original,ForecastedData,90)
    PIEvaluation.iloc[2,0], PIEvaluation.iloc[2,1], PIEvaluation.iloc[2,2], PIEvaluation.iloc[2,3]=find_Prediction_Interval_Evaluation(conf80,Original,ForecastedData,80)
    PIEvaluation.iloc[3,0], PIEvaluation.iloc[3,1], PIEvaluation.iloc[3,2], PIEvaluation.iloc[3,3]=find_Prediction_Interval_Evaluation(conf70,Original,ForecastedData,70)
    PIEvaluation.iloc[4,0], PIEvaluation.iloc[4,1], PIEvaluation.iloc[4,2], PIEvaluation.iloc[4,3]=find_Prediction_Interval_Evaluation(conf60,Original,ForecastedData,60)
    img_name=filename+"_"+modelname+"Exponential_Distribution_Prediction_Interval.png"
    plt.savefig(img_name)
    return IF,PIEvaluation

In [ ]:
def Laplace(Original,ForecastedData,error,filename,modelname,Scale):
    #Calculate different confidence intervals with Laplace Distribution
    location=0
    conf95=find_confidence_interval_laplace(0.95,location,Scale,error,ForecastedData)
    conf90=find_confidence_interval_laplace(0.90,location,Scale,error,ForecastedData)
    conf80=find_confidence_interval_laplace(0.80,location,Scale,error,ForecastedData)
    conf70=find_confidence_interval_laplace(0.70,location,Scale,error,ForecastedData)
    conf60=find_confidence_interval_laplace(0.60,location,Scale,error,ForecastedData)
    IF = pd.concat([conf95, conf90, conf80, conf70, conf60], axis=1)
    fig, ax = plt.subplots(1,1)
    fig.set_figheight(5)
    fig.set_figwidth(10)
    ax.fill_between(conf95.index,conf95.iloc[:,0],conf95.iloc[:,1], color='#ff0000',label='95%')
    ax.fill_between(conf90.index,conf90.iloc[:,0],conf90.iloc[:,1], color='#cf0000',label='90%')
    ax.fill_between(conf80.index,conf80.iloc[:,0],conf80.iloc[:,1], color='#af0000',label='80%')
    ax.fill_between(conf90.index,conf70.iloc[:,0],conf70.iloc[:,1], color='#8f0000',label='70%')
    ax.fill_between(conf80.index,conf60.iloc[:,0],conf60.iloc[:,1], color='#6f0000',label='60%')
    ax.plot(ForecastedData, linewidth=3,label='Forecasts')
    ax.plot(Original, linewidth=3,label='True Values',color='black', ls=':')
    ax.legend(loc="best")
    ax.set(xlabel='Time / Month', ylabel=filename)
    PIEvaluation=pd.DataFrame(np.zeros((5,4)))
    PIEvaluation.iloc[0,0], PIEvaluation.iloc[0,1], PIEvaluation.iloc[0,2], PIEvaluation.iloc[0,3]=find_Prediction_Interval_Evaluation(conf95,Original,ForecastedData,95)
    PIEvaluation.iloc[1,0], PIEvaluation.iloc[1,1], PIEvaluation.iloc[1,2], PIEvaluation.iloc[1,3]=find_Prediction_Interval_Evaluation(conf90,Original,ForecastedData,90)
    PIEvaluation.iloc[2,0], PIEvaluation.iloc[2,1], PIEvaluation.iloc[2,2], PIEvaluation.iloc[2,3]=find_Prediction_Interval_Evaluation(conf80,Original,ForecastedData,80)
    PIEvaluation.iloc[3,0], PIEvaluation.iloc[3,1], PIEvaluation.iloc[3,2], PIEvaluation.iloc[3,3]=find_Prediction_Interval_Evaluation(conf70,Original,ForecastedData,70)
    PIEvaluation.iloc[4,0], PIEvaluation.iloc[4,1], PIEvaluation.iloc[4,2], PIEvaluation.iloc[4,3]=find_Prediction_Interval_Evaluation(conf60,Original,ForecastedData,60)
    img_name=filename+"_"+modelname+"Laplace_Distribution_Prediction_Interval.png"
    plt.savefig(img_name)
    return IF,PIEvaluation

In [ ]:
def Maxwell(Original,ForecastedData,error,filename,modelname,Scale):
    #Calculate different confidence intervals with Maxwell Distribution
    location=0
    conf95=find_confidence_interval_maxwell(0.95,location,Scale,error,ForecastedData)
    conf90=find_confidence_interval_maxwell(0.90,location,Scale,error,ForecastedData)
    conf80=find_confidence_interval_maxwell(0.80,location,Scale,error,ForecastedData)
    conf70=find_confidence_interval_maxwell(0.70,location,Scale,error,ForecastedData)
    conf60=find_confidence_interval_maxwell(0.60,location,Scale,error,ForecastedData)
    IF = pd.concat([conf95, conf90, conf80, conf70, conf60], axis=1)
    fig, ax = plt.subplots(1,1)
    fig.set_figheight(5)
    fig.set_figwidth(10)
    ax.fill_between(conf95.index,conf95.iloc[:,0],conf95.iloc[:,1], color='#ff0000',label='95%')
    ax.fill_between(conf90.index,conf90.iloc[:,0],conf90.iloc[:,1], color='#cf0000',label='90%')
    ax.fill_between(conf80.index,conf80.iloc[:,0],conf80.iloc[:,1], color='#af0000',label='80%')
    ax.fill_between(conf90.index,conf70.iloc[:,0],conf70.iloc[:,1], color='#8f0000',label='70%')
    ax.fill_between(conf80.index,conf60.iloc[:,0],conf60.iloc[:,1], color='#6f0000',label='60%')
    ax.plot(ForecastedData, linewidth=3,label='Forecasts')
    ax.plot(Original, linewidth=3,label='True Values',color='black', ls=':')
    ax.legend(loc="best")
    ax.set(xlabel='Time / Month', ylabel=filename)
    PIEvaluation=pd.DataFrame(np.zeros((5,4)))
    PIEvaluation.iloc[0,0], PIEvaluation.iloc[0,1], PIEvaluation.iloc[0,2], PIEvaluation.iloc[0,3]=find_Prediction_Interval_Evaluation(conf95,Original,ForecastedData,95)
    PIEvaluation.iloc[1,0], PIEvaluation.iloc[1,1], PIEvaluation.iloc[1,2], PIEvaluation.iloc[1,3]=find_Prediction_Interval_Evaluation(conf90,Original,ForecastedData,90)
    PIEvaluation.iloc[2,0], PIEvaluation.iloc[2,1], PIEvaluation.iloc[2,2], PIEvaluation.iloc[2,3]=find_Prediction_Interval_Evaluation(conf80,Original,ForecastedData,80)
    PIEvaluation.iloc[3,0], PIEvaluation.iloc[3,1], PIEvaluation.iloc[3,2], PIEvaluation.iloc[3,3]=find_Prediction_Interval_Evaluation(conf70,Original,ForecastedData,70)
    PIEvaluation.iloc[4,0], PIEvaluation.iloc[4,1], PIEvaluation.iloc[4,2], PIEvaluation.iloc[4,3]=find_Prediction_Interval_Evaluation(conf60,Original,ForecastedData,60)
    img_name=filename+"_"+modelname+"Maxwell_Distribution_Prediction_Interval.png"
    plt.savefig(img_name)
    return IF,PIEvaluation

In [ ]:
#Deterministic Forecasting
for k in range(0,10,1):
    filename=''
    LagLength=1
    #0-3 leptokurtic TSD
    if k==0:
        filename='Alcohol.csv'
        LagLength=2
    elif k==1:
        filename='lynx.csv'
        LagLength=10
    elif k==2:
        filename='River.csv'
        LagLength=12
    elif k==3:
        filename='stock.csv'
        LagLength=2
    #4-7 Platykurtic TSD
    elif k==4:
        filename='NewWorkWater.csv'
        LagLength=2
    elif k==5:
        filename='PlasticContainerDemand.csv'
        LagLength=2
    elif k==6:
        filename='temp.csv'
        LagLength=12
    elif k==7:
        filename='ToothpasteMarketShare.csv'
        LagLength=2
    #Mesokurtic
    elif k==8:
        filename='chicken.csv'
        LagLength=12
    elif k==9:
        filename='IntTraUK.csv'
        LagLength=12
    #Read the Time Series Dataset
    Timeseries_Data=pd.read_csv(filename,header=None)
    h=1
    lt=Timeseries_Data.shape[0]
    lenTrain=int(round(lt*0.6))
    lenValidation=int(round(lt*0.2))
    lenTest=int(lt-lenTrain-lenValidation)
    # NORMALIZE THE DATA
    normalizedData=minmaxNorm(Timeseries_Data,lenTrain+lenValidation);
    # Transform the Time Series into Patterns Using Sliding Window
    X, y = get_Patterns(normalizedData, LagLength, h)
    Deterministic_MeanAccuracyT=pd.DataFrame()
    Deterministic_StdAccuracyT=pd.DataFrame()
    file1='./Deterministic_'+str(filename[:-4])+"_Accuracy.xlsx"
    file2='./Deterministic_'+str(filename[:-4])+"_Forecasts.xlsx"
    Deterministic_Forecasts=pd.DataFrame()
    Deterministic_Accuracy=pd.DataFrame()
    for j in range(0,6,1):
        # print('Model:::::::::::::::::::::::::::::::',j)
        if j==0:
            model=LinearRegression()
            name='1.LinearRegression'
        elif j==1:
            model=Lasso()
            name='2.Lasso'
        elif j==2:
            model=Ridge()
            name='3.Ridge'
        elif j==3:
            model=ElasticNet()
            name='4.ElasticNet'
        elif j==4:
            model=HuberRegressor()
            name='5.HuberRegressor'
        elif j==5:
            model=LinearSVR()
            name='6.LinearSVR'
        else:
            print('Completed.....................')
        ynorm1=Find_Fitness(X,y,lenValidation,lenTest,model)
        ynorm=pd.DataFrame(normalizedData.iloc[0:LagLength,0])
        ynorm=ynorm.append(ynorm1,ignore_index = True)
        yhat=minmaxDeNorm(Timeseries_Data, ynorm, lenTrain+lenValidation)

        Deterministic_Accuracy.loc[j,0],Deterministic_Accuracy.loc[j,1]=findRMSE( Timeseries_Data,yhat,lenTrain+lenValidation)
        Deterministic_Accuracy.loc[j,2],Deterministic_Accuracy.loc[j,3]=findSMAPE( Timeseries_Data,yhat,lenTrain+lenValidation)
        Deterministic_Accuracy.loc[j,4],Deterministic_Accuracy.loc[j,5]=findMAE( Timeseries_Data,yhat,lenTrain+lenValidation)
        Deterministic_Accuracy.loc[j,6],Deterministic_Accuracy.loc[j,7]=findMASE( Timeseries_Data,yhat,lenTrain+lenValidation)
        Deterministic_Forecasts=Deterministic_Forecasts.append(yhat.T,ignore_index = True)
    Deterministic_Accuracy.to_excel(file1,sheet_name='Accuracy',index=False)
    Deterministic_Forecasts.T.to_excel(file2,sheet_name='Forecasts',index=False)
    Deterministic_MeanAccuracy=pd.DataFrame(np.mean(Deterministic_Accuracy))
    Deterministic_MeanAccuracyT=Deterministic_MeanAccuracyT.append(Deterministic_MeanAccuracy.T, ignore_index = True)
    Deterministic_StdAccuracy=pd.DataFrame(np.std(Deterministic_Accuracy))
    Deterministic_StdAccuracyT=Deterministic_StdAccuracyT.append(Deterministic_StdAccuracy.T, ignore_index = True)
    del Deterministic_Accuracy
    del Deterministic_Forecasts
    del Deterministic_MeanAccuracy
    del Deterministic_StdAccuracy
Deterministic_MeanAccuracyT.to_excel('All_Model_Point_Mean_Accuracy.xlsx',sheet_name='All_Model_Point_Accuracy',index=False)
del Deterministic_MeanAccuracyT
Deterministic_StdAccuracyT.to_excel('All_Model_Point_Stdev_Accuracy.xlsx',sheet_name='All_Model_Point_Stdev_Accuracy',index=False)
del Deterministic_StdAccuracyT

In [ ]:
#Interval Forecasting
for k in range(0,10,1):
    filename='';
    Forecasts=pd.DataFrame()
    Accuracy=pd.DataFrame()
    #0-3 leptokurtic TSD
    if k==0:
        filename='Alcohol.csv'
        model=LinearSVR()
        LagLength=2
        Scale=1
    elif k==1:
        filename='lynx.csv'
        model=Ridge()
        LagLength=10
        Scale=100
    elif k==2:
        filename='River.csv'
        model=HuberRegressor()
        LagLength=12
        Scale=1
    elif k==3:
        filename='stock.csv'
        model=Ridge()
        LagLength=2
        Scale=5
    #4-7 Platykurtic TSD
    elif k==4:
        filename='NewWorkWater.csv'
        model=HuberRegressor()
        LagLength=2
        Scale=5
    elif k==5:
        filename='PlasticContainerDemand.csv'
        model=HuberRegressor()
        LagLength=2
        Scale=100
    elif k==6:
        filename='temp.csv'
        model=HuberRegressor()
        LagLength=12
        Scale=5
    elif k==7:
        filename='ToothpasteMarketShare.csv'
        model=HuberRegressor()
        LagLength=2
        Scale=0.5
    #Mesokurtic
    elif k==8:
        filename='chicken.csv'
        model=HuberRegressor()
        LagLength=12
        Scale=200
    elif k==9:
        filename='IntTraUK.csv'
        model=LinearRegression()
        LagLength=12
        Scale=200
    #Read the Time Series Dataset
    Original=pd.read_csv(filename,header=None)
    h=1
    lt=Original.shape[0]
    lenTrain=int(round(lt*0.6))
    lenValidation=int(round(lt*0.2))
    lenTest=int(lt-lenTrain-lenValidation)
    # NORMALIZE THE DATA
    normalizedData=minmaxNorm(Original,lenTrain+lenValidation);
    # Transform the Time Series into Patterns Using Sliding Window
    X, y = get_Patterns(normalizedData, LagLength, h)
    file1='./Probabilistic_'+str(filename[:-4])+"_Accuracy.xlsx"
    file2='./Probabilistic_'+str(filename[:-4])+"_Forecasts.xlsx"
    MeanAccuracyT=pd.DataFrame()
    StdAccuracyT=pd.DataFrame()
    file_name = filename[:-4]
    ynorm1=Find_Fitness(X,y,lenValidation,lenTest,model)
    ynorm=pd.DataFrame(normalizedData.iloc[0:LagLength,0])
    ynorm=ynorm.append(ynorm1,ignore_index = True)
    ForecastedData=minmaxDeNorm(Original, ynorm, lenTrain+lenValidation)
    error=Original-ForecastedData
    PIE=pd.DataFrame(np.zeros((5,4)))
    for d in range(10):
        if d==0:
            IF,PIE=gaussian(Original,ForecastedData,error,file_name,name[2:],Scale)
        elif d==1:
            IF,PIE=t_location_scale(Original,ForecastedData,error,file_name,name[2:],Scale)
        elif d==2:
            IF,PIE=Beta(Original,ForecastedData,error,file_name,name[2:],Scale)
        elif d==3:
            IF,PIE=Cauchy(Original,ForecastedData,error,file_name,name[2:],Scale)
        elif d==4:
            IF,PIE=Gamma(Original,ForecastedData,error,file_name,name[2:],Scale)
        elif d==5:
            IF,PIE=Rayleigh(Original,ForecastedData,error,file_name,name[2:],Scale)
        elif d==6:
            IF,PIE=Uniform(Original,ForecastedData,error,file_name,name[2:],Scale)
        elif d==7:
            IF,PIE=Exponential(Original,ForecastedData,error,file_name,name[2:],Scale)
        elif d==8:
            IF,PIE=Laplace(Original,ForecastedData,error,file_name,name[2:],Scale)
        elif d==9:
            IF,PIE=Maxwell(Original,ForecastedData,error,file_name,name[2:],Scale)
        else:
            print('Completed.....................')
        t=-1
        for q in range(20):
            if(q%5==0):
                t+=1
            Accuracy.loc[d,q]=PIE.loc[t,q%4]
        Forecasts=pd.concat([Forecasts,IF],axis=1)
    Accuracy.to_excel(file1,sheet_name='Accuracy',index=False)
    Forecasts.T.to_excel(file2,sheet_name='Forecasts',index=False)
    MeanAccuracy=pd.DataFrame(np.mean(Accuracy))
    MeanAccuracyT=MeanAccuracyT.append(MeanAccuracy.T, ignore_index = True)
    StdAccuracy=pd.DataFrame(np.std(Accuracy))
    StdAccuracyT=StdAccuracyT.append(StdAccuracy.T, ignore_index = True)
    del Accuracy
    # del Forecasts
    del MeanAccuracy
    del StdAccuracy
MeanAccuracyT.to_excel('All_Model_Mean_Accuracy.xlsx',sheet_name='All_Model_Accuracy',index=False)
del MeanAccuracyT
StdAccuracyT.to_excel('All_Model_Stdev_Accuracy.xlsx',sheet_name='All_Model_Stdev_Accuracy',index=False)
del StdAccuracyT
# print(Accuracy)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')